In [4]:
import pandas as pd
import numpy as np

# keepCols = ['state','state_postal','state_fips','state_icpsr','county_name','county_fips',#'county_lat','county_long',
#             'jurisdiction','precinct','votes','candidate_normalized','office','writein',
#             'party','mode']

keepCols = ['state_postal','state_fips','state_icpsr','county_name','jurisdiction',
            'precinct','county_fips','votes','candidate_normalized','party']
dtypes = {"state_postal":"object","states_fips":np.int64,"state_icpsr":np.int64,"county_name":"object","jurisdiction":str,
          "precinct":str,"county_fips":str,"candidate_normalized":"object","party":"object","votes":np.int64}

df = pd.read_csv('data\\2016-precinct-president.csv',encoding="ISO-8859-1", low_memory=False, usecols=keepCols, dtype=dtypes)
# print(df.info())
# print(df['mode'].unique())
df['county_fips']=df['county_fips'].apply(lambda x: '{0:0>5}'.format(x) if len(str(x)) < 5 else str(x))

# Missing county_fips for Alaska and Hawaii

In [6]:
# Assign new winner column, will update to correct value
df = df.assign(winner=np.full_like(df['candidate_normalized'], 'None', dtype=str))
df = df.assign(total_county_votes=np.full_like(df['candidate_normalized'], 0, dtype=np.int64))
df = df.assign(votes_normalized=np.full_like(df['candidate_normalized'], 0, dtype=np.float64))



grouped = df[['state_postal','county_fips','jurisdiction','precinct','candidate_normalized','votes']].groupby(by=['state_postal','county_fips','jurisdiction'])

for group in grouped:
    sub_df = group[1]
    
    tallies = []
    total_votes = 0
    
    # Loop through candidates
    candidates = sub_df['candidate_normalized'].unique()
    for _ in range(len(candidates)):
        candidate = candidates[_]
        subsub_df = sub_df.query("candidate_normalized==@candidate")
        tallies.append([str(candidate),int(subsub_df['votes'].sum())])
        total_votes += subsub_df['votes'].sum()

    # Calculate winner
    winner_index = max(range(len(tallies)), key=lambda i: tallies[i][1])
    winning_candidate = tallies[winner_index][0]
    
    # List of indices to change the 'winner' column
    indices = sub_df.index.tolist()

    df.loc[indices,['winner']] = winning_candidate
    df.loc[indices,['total_county_votes']] = total_votes
    try:
        df.loc[indices,['votes_normalized']] = sub_df['votes'][indices].values/total_votes
    except:
        print(sub_df['votes'][indices])
        print(total_votes)
        print(indices)
        break


<ipython-input-6-c3d24a7a266d>:35: RuntimeWarning: invalid value encountered in true_divide
  df.loc[indices,['votes_normalized']] = sub_df['votes'][indices].values/total_votes


# Write cleaned/prepared DataFrame to csv file

In [10]:
df.to_csv('data\\2016-precinct-president-prepared.csv')
print(df.info())

In [11]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1989234 entries, 0 to 1989233
Data columns (total 13 columns):
 #   Column                Dtype  
---  ------                -----  
 0   state_postal          object 
 1   state_fips            int64  
 2   state_icpsr           int64  
 3   county_name           object 
 4   county_fips           object 
 5   jurisdiction          object 
 6   precinct              object 
 7   candidate_normalized  object 
 8   party                 object 
 9   votes                 int64  
 10  winner                object 
 11  total_county_votes    int64  
 12  votes_normalized      float64
dtypes: float64(1), int64(4), object(8)
memory usage: 197.3+ MB
None
